In [1]:
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from core.elements.dormitory import Dormitory
from core.elements.student import Student
from core.elements.floor import Floor
from core.elements.room import Room

from frontend.pygame_manager import PygameManager

In [3]:
dormitory = Dormitory()

pygame_manager = PygameManager()

# Main game loop
clock = pygame.time.Clock()
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            running = pygame_manager.handle_keydown(event, dormitory)
    pygame_manager.draw(dormitory)
    clock.tick(10)

pygame.quit()